In [123]:
from ortools.sat.python import cp_model
import numpy as np
from typing import List, Dict, Tuple

In [124]:
import numpy as np
from typing import List, Dict
from ortools.sat.python import cp_model
import pandas as pd

class JobShopScheduler:
    def __init__(self, num_machines: int, num_jobs: int, num_categories: int):
        self.num_machines = num_machines
        self.num_jobs = num_jobs
        self.num_categories = num_categories
        self.model = cp_model.CpModel()
        
    def setup_problem(self, 
                     job_categories: List[int],
                     processing_times: np.ndarray,  # shape: (machines, categories)
                     arrival_times: List[int]):
        """
        Set up the job shop problem with given parameters.
        
        Args:
            job_categories: List of category indices for each job
            processing_times: 2D array where processing_times[m][c] is processing time 
                            for category c on machine m
            arrival_times: List of arrival times for each job
        """
        self.job_categories = job_categories
        self.processing_times = processing_times
        self.arrival_times = arrival_times
        
        # Decision variables
        self.start_times = []
        self.end_times = []
        self.machine_assignments = []
        
        # Create variables for each job
        for job_id in range(self.num_jobs):
            # Start time variable (must be >= arrival time)
            start_var = self.model.NewIntVar(arrival_times[job_id], 
                                           cp_model.INT32_MAX, 
                                           f'start_{job_id}')
            self.start_times.append(start_var)
            
            # Machine assignment variable
            machine_var = self.model.NewIntVar(0, self.num_machines - 1, 
                                             f'machine_{job_id}')
            self.machine_assignments.append(machine_var)
            
            # Processing time depends on machine and job category
            category = job_categories[job_id]
            processing_time_var = self.model.NewIntVar(0, cp_model.INT32_MAX, 
                                                     f'proc_time_{job_id}')
            
            # Create element constraint for processing time based on machine assignment
            machine_processing_times = [processing_times[m][category] 
                                      for m in range(self.num_machines)]
            self.model.AddElement(machine_var, machine_processing_times, 
                                processing_time_var)
            
            # End time variable
            end_var = self.model.NewIntVar(0, cp_model.INT32_MAX, f'end_{job_id}')
            self.model.Add(end_var == start_var + processing_time_var)
            self.end_times.append(end_var)
        
        # No-overlap constraint: jobs on the same machine cannot overlap
        self.setup_no_overlap_constraints()
        
        
        # Alternative objective: minimize maximum flow time (completion - arrival)
        self.flow_times = []
        for job_id in range(self.num_jobs):
            flow_time = self.model.NewIntVar(0, cp_model.INT32_MAX, f'flow_{job_id}')
            self.model.Add(flow_time == self.start_times[job_id] - arrival_times[job_id])
            self.flow_times.append(flow_time)
        
        self.max_flow_time = self.model.NewIntVar(0, cp_model.INT32_MAX, 'max_flow_time')
        self.model.AddMaxEquality(self.max_flow_time, self.flow_times)
        
        # Set objective to minimize maximum flow time
        self.model.Minimize(self.max_flow_time)
    
    def setup_no_overlap_constraints(self):
        """Set up no-overlap constraints for each machine using optional intervals."""
        machine_optional_intervals = [[] for _ in range(self.num_machines)]
        
        for job_id in range(self.num_jobs):
            category = self.job_categories[job_id]
            base_start = self.start_times[job_id]
            base_end = self.end_times[job_id]
            machine_var = self.machine_assignments[job_id]
            
            # Create one optional interval per machine for this job
            assignment_literals = []
            for machine in range(self.num_machines):
                # Boolean: is this job assigned to this machine?
                is_assigned = self.model.NewBoolVar(f'job_{job_id}_on_machine_{machine}')
                assignment_literals.append(is_assigned)
                
                self.model.Add(machine_var == machine).OnlyEnforceIf(is_assigned)
                self.model.Add(machine_var != machine).OnlyEnforceIf(is_assigned.Not())
                
                # Processing time for this (machine, category)
                proc_time = int(self.processing_times[machine][category])
                
                # Create optional interval variables
                opt_start = self.model.NewIntVar(0, cp_model.INT32_MAX, f'opt_start_{job_id}_m{machine}')
                opt_end = self.model.NewIntVar(0, cp_model.INT32_MAX, f'opt_end_{job_id}_m{machine}')
                
                # Link optional interval to global timing only if assigned
                self.model.Add(opt_start == base_start).OnlyEnforceIf(is_assigned)
                self.model.Add(opt_end == base_end).OnlyEnforceIf(is_assigned)
                
                # Optional interval variable
                opt_interval = self.model.NewOptionalIntervalVar(
                    opt_start, proc_time, opt_end, is_assigned,
                    f'opt_interval_{job_id}_m{machine}'
                )
                
                machine_optional_intervals[machine].append(opt_interval)
            
            # Ensure each job is assigned to exactly one machine
            self.model.Add(sum(assignment_literals) == 1)
        
        # Add no-overlap constraint for each machine
        for machine in range(self.num_machines):
            if machine_optional_intervals[machine]:
                self.model.AddNoOverlap(machine_optional_intervals[machine])
    
    def solve(self, time_limit: int = 300) -> Dict:
        """Solve the problem and return results."""
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = time_limit
        solver.parameters.num_search_workers = 8  # Use multiple cores
        
        status = solver.Solve(self.model)
        
        results = {
            'status': status,
            'solver': solver,
            'assignments': [],
            'schedule': [],
            'objective_value': None
        }
        
        if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
            results['objective_value'] = solver.Value(self.max_flow_time)
            
            # Collect assignment and timing information
            for job_id in range(self.num_jobs):
                machine = solver.Value(self.machine_assignments[job_id])
                start = solver.Value(self.start_times[job_id])
                end = solver.Value(self.end_times[job_id])
                flow_time = solver.Value(self.flow_times[job_id])
                
                results['assignments'].append({
                    'job_id': job_id,
                    'category': self.job_categories[job_id],
                    'machine': machine,
                    'start_time': start,
                    'end_time': end,
                    'flow_time': flow_time,
                    'arrival_time': self.arrival_times[job_id],
                    'processing_time': self.processing_times[machine][self.job_categories[job_id]]
                })
                
                results['schedule'].append({
                    'job': job_id,
                    'machine': machine,
                    'start': start,
                    'end': end
                })
        
        return results

def generate_sample_data():
    """Generate sample data for testing."""
    np.random.seed(42)
    
    num_machines = 8
    num_jobs = 100
    num_categories = 3
    
    # Job categories: randomly assign jobs to categories
    job_categories = np.random.randint(0, num_categories, num_jobs)
    
    # Processing times: machines have different speeds for different categories
    processing_times = np.array([
    [35, 15, 5],  # Machine 0
    [25, 15, 10],  # Machine 1
    [30, 17, 7],  # Machine 2
    [30, 13, 5],  # Machine 3
    [40, 15, 5],  # Machine 4
    [20, 15, 4],  # Machine 5
    [30, 20, 8],  # Machine 6
    [30, 10, 5],  # Machine 7
    ])
    

    # Arrival times: jobs arrive at different times
    arrival_times = []
    for i in range(num_jobs):
        if i < 20:
            arrival_times.append(1)
        elif i < 40:
            arrival_times.append(45)
        elif i < 60:
            arrival_times.append(90)
        elif i < 80:
            arrival_times.append(135)
        else:
            arrival_times.append(170)
    
    return num_machines, num_jobs, num_categories, job_categories, processing_times, arrival_times
def generate_std_3_data():
    """Generate sample data for testing."""
    np.random.seed(42)
    
    num_machines = 8
    num_jobs = 100
    num_categories = 3
    
    # Job categories: randomly assign jobs to categories
    job_categories = np.random.randint(0, num_categories, num_jobs)
    
    # Processing times: machines have different speeds for different categories
    processing_times = np.array([
    [30, 15, 5],  # Machine 0
    [30, 15, 5],  # Machine 1
    [30, 15, 5],  # Machine 2
    [30, 15, 5],  # Machine 3
    [30, 15, 5],  # Machine 4
    [30, 15, 5],  # Machine 5
    [30, 15, 5],  # Machine 6
    [30, 15, 5],  # Machine 7
    ])
    

    # Arrival times: jobs arrive at different times
    arrival_times = []
    for i in range(num_jobs):
        if i < 20:
            arrival_times.append(1)
        elif i < 40:
            arrival_times.append(45)
        elif i < 60:
            arrival_times.append(90)
        elif i < 80:
            arrival_times.append(135)
        else:
            arrival_times.append(170)
    
    return num_machines, num_jobs, num_categories, job_categories, processing_times, arrival_times

def generate_comparison_data():
    """Generate sample data for testing."""
    np.random.seed(42)
    
    num_machines = 8
    num_jobs = 100
    num_categories = 3
    
    # Job categories: randomly assign jobs to categories
    job_categories = np.random.randint(0, num_categories, num_jobs)
    
    # Processing times: machines have different speeds for different categories
    processing_times = np.array([
    [30, 15, 15],  # Machine 0
    [30, 15, 15],  # Machine 1
    [30, 15, 15],  # Machine 2
    [30, 15, 15],  # Machine 3
    [30, 15, 15],  # Machine 4
    [30, 15, 15],  # Machine 5
    [30, 15, 15],  # Machine 6
    [30, 15, 15],  # Machine 7
    ])
    

    # Arrival times: jobs arrive at different times
    arrival_times = []
    for i in range(num_jobs):
        if i < 20:
            arrival_times.append(1)
        elif i < 40:
            arrival_times.append(45)
        elif i < 60:
            arrival_times.append(90)
        elif i < 80:
            arrival_times.append(135)
        else:
            arrival_times.append(170)
    
    return num_machines, num_jobs, num_categories, job_categories, processing_times, arrival_times


def results_to_dataframe(results: Dict) -> pd.DataFrame:
    """Convert scheduling results to a pandas DataFrame."""
    if results['status'] not in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        return pd.DataFrame()  # Return empty DataFrame if no solution
    
    # Create DataFrame from assignments
    df = pd.DataFrame(results['assignments'])
    
    # Add makespan and solve time as metadata (optional)
    df.attrs['objective_value'] = results['objective_value']
    df.attrs['solve_time'] = results['solver'].WallTime()
    df.attrs['status'] = results['solver'].StatusName()
    
    return df

def print_schedule(results: Dict):
    """Print the scheduling results using pandas DataFrame."""
    df = results_to_dataframe(results)
    
    if df.empty:
        print("No solution found")
        return
    
    print(f"Solution found! Status: {df.attrs['status']}")
    print(f"Maximum flow time: {df.attrs['objective_value']}")
    print(f"Solve time: {df.attrs['solve_time']:.2f} seconds")
    
    # Print job assignments
    print("\nJob Assignments:")
    print(df[['job_id', 'category', 'machine', 'arrival_time', 'start_time', 'end_time', 'flow_time']]
          .sort_values('start_time')
          .to_string(index=False))
    
    # Print machine utilization
    print("\nMachine Utilization:")
    machine_stats = df.groupby('machine').agg(
        num_jobs=('job_id', 'count'),
        total_work=('processing_time', 'sum'),
        makespan=('end_time', 'max')
    ).reset_index()
    
    machine_stats['utilization_pct'] = np.where(
        machine_stats['makespan'] > 0,
        (machine_stats['total_work'] / machine_stats['makespan']) * 100,
        0
    )
    
    for _, row in machine_stats.sort_values('machine').iterrows():
        print(f"Machine {int(row['machine'])}: {int(row['num_jobs'])} jobs, "
              f"utilization: {row['utilization_pct']:.1f}%")
    
    df.to_csv("solnDynamic.csv")


# Example usage
if __name__ == "__main__":
    # Generate sample data
    #num_machines, num_jobs, num_categories, job_categories, processing_times, arrival_times = generate_sample_data()
    #num_machines, num_jobs, num_categories, job_categories, processing_times, arrival_times = generate_std_3_data()
    num_machines, num_jobs, num_categories, job_categories, processing_times, arrival_times = generate_comparison_data()
    
    # Create and solve the problem
    scheduler = JobShopScheduler(num_machines, num_jobs, num_categories)
    scheduler.setup_problem(job_categories, processing_times, arrival_times)
    
    # Solve with 5-minute time limit
    results = scheduler.solve(time_limit=60)
    
    # Print results
    print_schedule(results)
    
    
    # You can also access the raw results for further analysis
    if results['objective_value'] is not None:
        print(f"\nBest solution found with max flow time: {results['objective_value']}")

Solution found! Status: FEASIBLE
Maximum flow time: 56
Solve time: 60.03 seconds

Job Assignments:
 job_id  category  machine  arrival_time  start_time  end_time  flow_time
     11         2        2             1           1        16          0
      1         0        5             1           1        31          0
     15         0        0             1           1        31          0
      4         0        4             1           1        31          0
     12         0        6             1           1        31          0
      6         2        3             1           1        16          0
      7         1        7             1           1        16          0
     10         2        1             1           1        16          0
     18         1        3             1          16        31         15
      0         2        7             1          16        31         15
      5         0        2             1          16        46         15
      2      